In [1]:
# https://github.com/erhwenkuo/deep-learning-with-keras-notebooks/blob/master/1.8-seq2seq-introduction.ipynb

![title](./10minutes_seq2seq.PNG)

## Import

In [2]:
from keras.models import Sequential
from keras import layers
from keras.utils import plot_model
import numpy as np
from six.moves import range
from IPython.display import Image

Using TensorFlow backend.


In [3]:
class CharacterTable(object):
    """
    給予一組的字符:
    + 將這些字符使用one-hot編碼成數字表示
    + 解碼one-hot編碼數字表示成為原本的字符
    + 解碼字符機率的向量以回覆最有可能的字符
    """
    def __init__(self, chars):
        """初始化字符表
        
        # 參數:
            chars: 會出現在輸入的可能字符集
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
        
    def encode(self, C, num_rows):
        """對輸入的字串進行one-hot編碼
        
        # 參數:
            C: 要被編碼的字符
            num_rows: one-hot編碼後要回傳的最大行數。這是用來確保每一個輸入都會得到
            相同行數的輸出
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        """對輸入的編碼(向量)進行解碼
        
        # 參數:
            x: 要被解碼的字符向量或字符編碼
            calc_argmax: 是否要用argmax算符找出機率最大的字符編碼
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)
    
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


## 相關的參數與產生訓練用的資料集

In [4]:
# 模型與資料集的參數
TRAINING_SIZE = 50000 # 訓練資料集的samples數
DIGITS = 3            # 加數或被加數的字符數
INVERT = True 

# 輸入的最大長度 'int + int' (比如, '345+678')
MAXLEN = DIGITS + 1 + DIGITS

# 所有要用到的字符(包括數字、加號及空格)
chars = '0123456789+ '
ctable = CharacterTable(chars) # 創建CharacterTable的instance

questions = [] # 訓練用的句子 "xxx+yyy"
expected = []  # 訓練用的標籤
seen = set()

print('Generating data...') # 產生訓練資料

Generating data...


In [5]:
f = lambda: int(''.join(np.random.choice(list('0123456789'))
                           for i in range(np.random.randint(1, DIGITS+1))))
f()

17

In [6]:
while len(questions) < TRAINING_SIZE:
    # 數字產生器 (3個字符)
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                           for i in range(np.random.randint(1, DIGITS+1))))
    a, b = f(), f()
    # 跳過己經看過的題目以及x+Y = Y+x這樣的題目
    key = tuple(sorted((a, b)))  # 排序a, b
    if key in seen:
        continue    
    seen.add(key)
    
    # 當數字不足MAXLEN則填補空白
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    
    # 答案的最大的字符長度為DIGITS + 1 (3位數+3位數 <= 4位數)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        # 故意轉的~~~~
        # 調轉問題字符的方向, 比如. '12+345'變成'543+21'
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
    
print('Total addition questions:', len(questions))

Total addition questions: 50000


## 資料的前處理

In [26]:
# 把資料做適當的轉換, LSTM預期的資料結構 -> [samples, timesteps, features]
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool) # 初始一個3維的numpy ndarray (特徵資料)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool) # 初始一個3維的numpy ndarray (標籤資料)
print(len(questions))
print(MAXLEN)
print(DIGITS + 1)  # 答案最大四位數
print(len(chars))
print(y.shape)
print(y)

Vectorization...
50000
7
4
12
(50000, 4, 12)
[[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 ...

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False 

In [27]:
# 將"特徵資料"轉換成LSTM預期的資料結構 -> [samples, timesteps, features]
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)      # <--- 要了解為什麼要這樣整理資料

print("Feature data: ", x.shape)

# 將"標籤資料"轉換成LSTM預期的資料結構 -> [samples, timesteps, features]
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)  # <--- 要了解為什麼要這樣整理資料

print("Label data: ", y.shape)

Feature data:  (50000, 7, 12)
Label data:  (50000, 4, 12)


In [28]:
# 打散 Shuffle(x, y)
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# 保留10%的資料來做為驗證
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


## 構建網絡架構

In [30]:
# 可以試著替代其它種的rnn units, 比如,GRU或SimpleRNN
LSTM = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
model = Sequential()

# ===== 編碼 (encoder) ====

# 使用RNN“編碼”輸入序列，產生HIDDEN_SIZE的輸出。
# 注意：在輸入序列長度可變的情況下，使用input_shape =（None，num_features）
model.add(LSTM(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars)))) # MAXLEN代表是timesteps, 而len(chars)是one-hot編碼的features

# 作為解碼器RNN的輸入，重複提供每個時間步的RNN的最後一個隱藏狀態(我應該會看成time step)。
# 重複“DIGITS + 1”次，因為這是最大輸出長度，例如當DIGITS = 3時，最大輸出是999 + 999 = 1998（長度為4)。
model.add(layers.RepeatVector(DIGITS + 1))  # 決定time step，也就是輸出幾個字

# ==== 解碼 (decoder) ====
# 解碼器RNN可以是多層堆疊或單層。
for _ in range(LAYERS):
    # 通過將return_sequences設置為True，不僅返回最後一個輸出，而且還以（num_samples，timesteps，output_dim）
    # 的形式返回所有輸出。這是必要的，因為下面的TimeDistributed需要第一個維度是時間步長。
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))

# 對輸入的每個時間片推送到密集層來對於輸出序列的每一時間步，決定選擇哪個字符。
model.add(layers.TimeDistributed(layers.Dense(len(chars))))  # 每個字的one hot維度

model.add(layers.Activation('softmax'))  # 每一個one hot都給一個softmax
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_2 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


## 訓練模型/驗證評估
我們將進行50次的訓練，並且在每次訓練之後就進行檢查。

In [31]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(45000, 7, 12)
(45000, 4, 12)
(5000, 7, 12)
(5000, 4, 12)


In [49]:
for iteration in range(1, 30):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
             batch_size=BATCH_SIZE,
             epochs=1,
             validation_data=(x_val, y_val))
    
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        
        q = ctable.decode(rowx[0]) # question: x
        correct = ctable.decode(rowy[0]) # answer: y
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q:', q[::-1] if INVERT else q, end=' ')  # 印出來的時候要倒回來
        print('A:', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 185us/step - loss: 1.8910 - acc: 0.3202 - val_loss: 1.7940 - val_acc: 0.3417
Q 501+5   T 506  ☒ 11  
Q 862+5   T 867  ☒ 101 
Q 38+386  T 424  ☒ 108 
Q 223+469 T 692  ☒ 103 
Q 841+6   T 847  ☒ 10  
Q 153+804 T 957  ☒ 108 
Q 11+741  T 752  ☒ 101 
Q 533+195 T 728  ☒ 108 
Q 9+851   T 860  ☒ 101 
Q 155+976 T 1131 ☒ 111 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 148us/step - loss: 1.7381 - acc: 0.3585 - val_loss: 1.6695 - val_acc: 0.3754
Q 589+35  T 624  ☒ 900 
Q 964+14  T 978  ☒ 104 
Q 1+51    T 52   ☒ 11  
Q 909+2   T 911  ☒ 100 
Q 82+172  T 254  ☒ 332 
Q 577+408 T 985  ☒ 104 
Q 832+12  T 844  ☒ 332 
Q 3+385   T 388  ☒ 332 
Q 286+5   T 291  ☒ 660 
Q 0+682   T 682  ☒ 172 

-------------------

45000/45000 [==============================] - 7s 150us/step - loss: 0.2509 - acc: 0.9393 - val_loss: 0.2201 - val_acc: 0.9477
Q 700+435 T 1135 ☒ 1145
Q 41+491  T 532  ☑ 532 
Q 6+367   T 373  ☑ 373 
Q 870+7   T 877  ☑ 877 
Q 173+178 T 351  ☒ 341 
Q 522+359 T 881  ☑ 881 
Q 215+344 T 559  ☒ 569 
Q 609+16  T 625  ☑ 625 
Q 447+924 T 1371 ☑ 1371
Q 545+63  T 608  ☑ 608 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 7s 148us/step - loss: 0.1781 - acc: 0.9644 - val_loss: 0.1681 - val_acc: 0.9597
Q 114+943 T 1057 ☒ 1056
Q 37+575  T 612  ☑ 612 
Q 73+621  T 694  ☑ 694 
Q 571+786 T 1357 ☑ 1357
Q 908+52  T 960  ☑ 960 
Q 672+51  T 723  ☑ 723 
Q 59+542  T 601  ☒ 501 
Q 741+42  T 783  ☑ 783 
Q 60+377  T 437  ☑ 437 
Q 747+23  T 770  ☑ 770 

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=====

我們可以看到在30次的訓練循環之後,我們己經可以在驗證準確性上達到99.8%的程度。

以上方法的一個先行條件是它假設:給定固定長度的序列當輸入[... t]有可能生成固定長度的目標[...t]序列。

這在某些情況下可行，但不適用於大多數使用情境。

## 一般情境：序列到序列(seq-to-seq)的典型範例
在一般情況下，輸入序列和輸出序列具有不同的長度（例如機器翻譯），並且為了開始預測目標，需要整個輸入序列。這需要更高級的設置，這是人們在沒有更多的上下文的情況下提到“序列到序列模型”時經常提到的。這是如何工作的：

1. RNN層（或多個RNN層的堆疊）作為“編碼器(encoder)”：它處理輸入序列並返回其自身的內部狀態。請注意，我們丟棄編碼器RNN的輸出，只保留它的內部狀態。這個狀態將作為下一步解碼器的“上下文”或“條件”。
2. 另一個RNN層（或多個RNN層的堆疊）充當“解碼器(decoder)”：對給定的目標序列的先前字符進行訓練，以預測目標序列的下一個字符。具體而言，訓練是將目標序列轉換成相同的序列偏移(offset)一個步驟的過程，這種情況稱為“教師強制(teacher forcing)”的訓練過程。重要的是，解碼器(decoder)使用來自編碼器(encoder)的狀態向量作為初始狀態，這是解碼器如何獲得關於它應該產生何種產出的關鍵資訊。實際上，解碼器學習以輸入序列為條件生成給定目標[... t]的目標[t + 1 ...]。

![title](./seq2se1_1.PNG)

在預測模式下，當我們想要解碼(decode)未知的輸入序列時，我們經歷一個稍微不同的過程：

1. 將輸入序列編碼成狀態向量(hidden state vector)。
2. 從大小為1的目標序列開始（只是開始序列字符）。
3. 將狀態向量和1-char目標序列饋送給解碼器以產生下一個字符的預測。
4. 使用這些預測對下一個字符進行採樣（我們簡單地使用argmax）。
5. 將採樣的字符附加到目標序列。
6. 重複，直到我們拿到生成序列結束字符或我們達到字符限制。

![title](./seq2se1_2.PNG)


也可以使用相同的過程來訓練Seq2Seq網絡，而不需要“教師強制”，即通過將解碼器的預測重新輸入到解碼器中。

讓我們用實際的程式碼來說明這些想法。

為了實現我們的範例，我們將使用英語句子對應的中文語句翻譯的數據集，您可以從<a href="http://www.manythings.org/anki/">[manythings.org/anki]下載這些數據集。</a> 要下載的文件被稱為cmn-eng.zip(簡中對應到英文)。為了更貼近學習的效果, 我己經把簡中轉成了繁中的版本（cmn-tw.txt），可以從<a href="https://github.com/erhwenkuo/deep-learning-with-keras-notebooks/blob/master/assets/data/cmn-tw.txt">Github</a>上取得這個資料檔。我們將實現一個字符級(character-level)的序列到序列模型，逐個字符地處理輸入，並逐個字符地產生輸出。另一個選擇是一個字級(word-level)模型，這個模型往往是機器翻譯更常見的。在這篇文章的最後，你會發現一些關於使用嵌入圖層(embedding layers)將我們的模型轉換為字級模型的參考連結。

### 資料準備
從<a href="https://github.com/erhwenkuo/deep-learning-with-keras-notebooks/blob/master/assets/data/cmn-tw.txt">Github</a>下載cmn-tw.txt檔案。
在這個Jupyter Notebook所在的目錄下產生一個新的子目錄"data"。
把下載的資料檔複製到"data"的目錄裡頭。
最後你的目錄結構看起來像這樣:


xxx.ipynb
data/   
└── cmn-tw.txt


以下是我們的流程總結：

1. 將句子(sentence)轉換為3個Numpy數組, encoder_input_data, decoder_input_data, decoder_target_data：

 - encoder_input_data是包含英文句子的one-hot向量化的三維形狀數組（num_pairs, max_english_sentence_length, num_english_characters）。
 - decoder_input_data是包含中文句子的one-hot向量化的三維形狀數組（num_pairs, max_chinese_sentence_length, num_chinese_characters）。
 - decoder_target_data與decoder_input_data相同，但是偏移了一個時間步長。 decoder_target_data [:,t,：]將與decoder_input_data [：,t+1,：]相同。
2. 訓練一個基本的基於LSTM的Seq2Seq模型來預測給出encoder_input_data和decoder_input_data的decoder_target_data。我們的模型使用教師強制(teacher forcing)的手法。

3. 解碼一些句子以檢查模型是否正常工作（將來自encoder_input_data的樣本轉換為來自decoder_target_data的對應樣本）。

整個網絡的架構構建可以參考以下的圖示:

![title](./seq2se1_MT.PNG)

### 引入相關的函數庫

In [3]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import os

# 專案的根目錄路徑
ROOT_DIR = os.getcwd()

# 置放訓練資料的目錄
DATA_PATH = os.path.join(ROOT_DIR, "data")

# 訓練資料檔
DATA_FILE = os.path.join(DATA_PATH, "cmn-tw.txt")

Using TensorFlow backend.


###  相關的參數

In [7]:
batch_size = 64 # 訓練時的批次數量
epochs = 100 # 訓練循環數
latent_dim = 256 # 編碼後的潛在空間的維度(dimensions of latent space)
num_samples = 10000 # 用來訓練的樣本數

### 資料的前處理

In [9]:
# 資料向量化
input_texts = []
target_texts = []
input_characters = set() # 英文字符集
target_characters = set() # 中文字符集

In [17]:
lines = open(DATA_FILE, mode="r", encoding="utf-8").read().split('\n')
lines

['Hi.\t嗨。',
 'Hi.\t你好。',
 'Run.\t你用跑的。',
 'Wait!\t等等！',
 'Hello!\t你好。',
 'I try.\t讓我來。',
 'I won!\t我贏了。',
 'Oh no!\t不會吧。',
 'Cheers!\t乾杯!',
 'He ran.\t他跑了。',
 'Hop in.\t跳進來。',
 'I lost.\t我迷失了。',
 'I quit.\t我退出。',
 "I'm OK.\t我沒事。",
 'Listen.\t聽著。',
 'No way!\t不可能！',
 'No way!\t沒門！',
 'Really?\t你確定？',
 'Try it.\t試試吧。',
 'We try.\t我們來試試。',
 'Why me?\t為什麼是我？',
 'Ask Tom.\t去問湯姆。',
 'Be calm.\t冷靜點。',
 'Be fair.\t公平點。',
 'Be kind.\t友善點。',
 'Be nice.\t和氣點。',
 'Call me.\t聯繫我。',
 'Call us.\t聯繫我們。',
 'Come in.\t進來。',
 'Get Tom.\t找到湯姆。',
 'Get out!\t滾出去！',
 'Go away!\t走開！',
 'Go away!\t滾！',
 'Go away.\t走開！',
 'Goodbye!\t再見！',
 'Goodbye!\t告辭！',
 'Hang on!\t等一下！',
 'He came.\t他來了。',
 'He runs.\t他跑。',
 'Help me.\t幫我一下。',
 'Hold on.\t堅持。',
 'Hug Tom.\t抱抱湯姆！',
 'I agree.\t我同意。',
 "I'm ill.\t我生病了。",
 "I'm old.\t我老了。",
 "It's OK.\t沒關係。",
 "It's me.\t是我。",
 'Join us.\t來加入我們吧。',
 'Keep it.\t留著吧。',
 'Kiss me.\t吻我。',
 'Perfect!\t完美！',
 'See you.\t再見！',
 'Shut up!\t閉嘴！',
 'Skip it.\t不管它。',
 'Take it.\t拿走吧。',


In [18]:
# 逐行的讀取與處理
for line in lines[: min(num_samples, len(lines)-1)]:
    input_text, target_text = line.split('\t')
    
    # 我們使用“tab”作為“開始序列[SOS]”字符或目標，“\n”作為“結束序列[EOS]”字符。 <-- **重要
    target_text = '\t' + target_text + '\n'
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [20]:
input_characters = sorted(list(input_characters)) # 全部輸入的字符集
target_characters = sorted(list(target_characters)) # 全部目標字符集

num_encoder_tokens = len(input_characters) # 所有輸入字符的數量
num_decoder_tokens = len(target_characters) # 所有輸目標字符的數量

max_encoder_seq_length = max([len(txt) for txt in input_texts]) # 最長的輸入句子長度
max_decoder_seq_length = max([len(txt) for txt in target_texts]) # 最長的目標句子長度

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 10000
Number of unique input tokens: 73
Number of unique output tokens: 2165
Max sequence length for inputs: 33
Max sequence length for outputs: 22


In [25]:
# 輸入字符的索引字典
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])

# 輸目標字符的索引字典
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

# 包含英文句子的one-hot向量化的三維形狀數組（num_pairs，max_english_sentence_length，num_english_characters）
encoder_input_data = np.zeros( # 10000, 33, 73
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

# 包含中文句子的one-hot向量化的三維形狀數組（num_pairs，max_chinese_sentence_length，num_chinese_characters）
decoder_input_data = np.zeros( # 10000, 22, 2165
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

# decoder_target_data與decoder_input_data相同，但是偏移了一個時間步長。 
# decoder_target_data [:， t，：]將與decoder_input_data [：，t + 1，：]相同
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [29]:
print(input_texts[:6], target_texts[:6])

['Hi.', 'Hi.', 'Run.', 'Wait!', 'Hello!', 'I try.'] ['\t嗨。\n', '\t你好。\n', '\t你用跑的。\n', '\t等等！\n', '\t你好。\n', '\t讓我來。\n']


In [30]:
# 把資料轉換成要用來訓練用的張量資料結構 <-- 重要
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        # 10000, 33, 73
        encoder_input_data[i, t, input_token_index[char]] = 1.
        
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep  --> 希望答案就是下一個字
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.